# CHAPTER 2 시퀀스

* 파이썬은 시퀀스를 단일하게 처리하는 ABC의 특징을 물려받았기 때문에 문자열, 리스트, 바이트 시퀀스, 배열 등등에 모두 반복, 슬라이싱, 정렬, 연결 등 공통된 연산을 적용할 수 있다.

**공통 시퀀스 연산**
![./ch_2_1](./ch2_1.jpg)
출처 : https://kongdols-room.tistory.com/24

# 2.1 내장 시퀀스 개요

파이썬 표준 라이브러리의 제공 시퀀스
* **컨테이너 시퀀스**
    * 서로 다른 자료형의 항목들을 담을 수 있음.
    * list, tuple, collections.deque 형.
    * 객체에 대한 참조를 담고 있으며 객체는 어떤 자료형도 될 수 있음.
      
      
* **균일 시퀀스**
    * 단 하나의 자료형만 담을 수 있음.
    * str, bytes, bytearray, memoryview, array.array 형.
    * 객체에 대한 참조 대신 자신의 메모리 공간에 각 항목의 값을 직접 담기 때문에 메모리를 더 적게 사용함.
    * 하지만, 문자, 바이트, 숫자 등 기본적인 자료형만 저장할 수 있음.

가변성에 따른 분류
* **가변 시퀀스**
    * list, bytearray, array.array, collections.deque, memoryview 형.
    
    
* **불변 시퀀스**
    * tuple, str, bytes 형.

# 2.2 지능형 리스트와 제너레이터 표현식

***지능형 리스트나 제너레이터 표현식을 사용하면 시퀀스를 간단히 생성할뿐만 아니라, 가독성이 좋고 실행 속도도 빠르다!***

In [1]:
# 예제 2-1 문자열에서 유니코드 코드포인트 리스트 만들기(버전 1)
symbols = '$¢£¥€¤'
codes = []
for symbol in symbols:
    codes.append(ord(symbol))
    
codes

[36, 162, 163, 165, 8364, 164]

In [2]:
# 예제 2-2 문자열에서 유니코드 코드포인트 리스트 만들기(버전 2)
symbols = '$¢£¥€¤'
codes = [ord(symbol) for symbol in symbols]
codes

[36, 162, 163, 165, 8364, 164]

* 지능형 리스트(2-2)는 의도를 명확히 보여준다. 
* 왜 (2-1)은 의도가 명확하지 않을까? 
    * **-> for 루프는 시퀀스를 읽어서 개수를 세거나 어떤 항목을 골라내는 등 다양한 일에 사용할 수 있기 때문이다.**
* 지능형 리스트는 오로지 새로운 리스트를 만드는 일만 한다.
* 주의) 생성된 리스트를 사용하지 않을 것이라면, 지능형 리스트를 사용하지 말아야 한다.
    * **-> 지능형 리스트를 남용해서 코드의 간결함을 잃을 수 있기 때문.**
* 지능형 리스트 구문이 두 줄 이상 넘어가는 경우에는 코드를 분할하거나 for문을 이용해서 작성하는 것이 더 낫다.
    * **-> 파이썬에서는 [], {}, () 안에서의 개행이 무시되기 떄문에 역슬래시를 사용하지 않고도 여러 줄에 걸쳐 리스트, 지능형 리스트, 제너레이터 표현식, 딕셔너리를 사용할 수 있다.**

In [3]:
x = 'ABC'
dummy = [ord(x) for x in x]

print(x) # x의 값이 유지된다.
print(dummy) # 지능형 리스트가 기대했던 리스트를 만든다.

ABC
[65, 66, 67]


Python3에서는 제너레이터 표현식(이하 Genexp), 지능형 리스트(이하 Listcomp), 지능형 집합(set comprehension), 지능형 딕셔너리(dict comprehension)은 함수처럼 **고유한 지역 범위**를 가진다.

-> 따라서 표현식 안에서 할당된 변수는 지역 변수지만, 주변 범위의 변수를 여전히 참조할 수 있다!

## 2.2.2 지능형 리스트와 map()/filter() 비교

`map()`과 `filter()`에 람다(lambda)를 이용해서 리스트를 구현할 수도 있지만 속도가 빠르지는 않다.

map(func, iter)  
filter(func, iter)

In [6]:
# 예제 2-3 지능형 리스트와 맵/필터 구성으로 만든 동일 리스트
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
(beyond_ascii)

[162, 163, 165, 8364, 164]

In [7]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]

**속도를 비교해보자!**

In [26]:
# 예제 코드 02-array-seq - listcomp_speed.py
import timeit

TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *('{:.3f}'.format(x) for x in res))
    
clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')
clock('listcomp + func :', '[ord(s) for s in symbols if non_ascii(ord(s))]')
clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')
clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')
# clock('listcomp + func :', '[filter(non_ascii, map(ord, symbols))]') 잘못된 방법!

listcomp        : 0.010 0.010 0.010 0.009 0.011
listcomp + func : 0.014 0.015 0.014 0.014 0.014
filter + lambda : 0.013 0.012 0.013 0.013 0.012
filter + func   : 0.012 0.012 0.012 0.012 0.012


## 2.2.3 데카르트 곱

Listcomp는 두 개 이상의 시퀀스 자료형의 데카르트 곱을 나타내는 리스트를 만들 수 있다.

In [28]:
# 예제 2-4 지능형 리스트를 이용한 데카르트 곱
colors = ['black', 'white']
sizes = ['S', 'M', 'L']

In [29]:
# 튜플 리스트를 생성해서 만들기
tshirts = [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [30]:
# []안에서는 역슬래시가 필요없다!
tshirts = [(color, size) for size in sizes
                         for color in colors]
tshirts

[('black', 'S'),
 ('white', 'S'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

## 2.2.4 제너레이터 표현식

* 튜플, 배열 등의 시퀀스형을 초기화하려면 listcomp를 이용할 수 있다.
* 하지만, 다른 생성자에 전달할 리스트를 통째로 만들지 않고 반복자 프로토콜(iterator protocol)을 이용해서 항목을 하나씩 생성하는 genexp은 메모리를 더 적게 사용한다.

In [31]:
# 예제 2-5
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

(36, 162, 163, 165, 8364, 164)

In [33]:
import array
array.array('I', (ord(symbol) for symbol in symbols))
# 배열 생성자의 첫번째 인수는 배열에 들어갈 숫자들을 저장할 자료형을 지정해야한다.

array('I', [36, 162, 163, 165, 8364, 164])

In [36]:
array.array('2', (ord(symbol) for symbol in symbols))

ValueError: bad typecode (must be b, B, u, h, H, i, I, l, L, q, Q, f or d)

In [37]:
# 예제 2-6 제너레이터 표현식에서의 데카르트 곱
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirts in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirts)

black S
black M
black L
white S
white M
white L


위 결과에서 볼 수 있듯이, genexp는 한 번에 하나의 항목을 생성하며, 6개의 티셔츠 종류를 담고 있는 리스트는 만들지 않는다.

###  2.2 정리
* Listcomp나 genexp는 리스트를 생성함에 있어서 가독성이 좋을 뿐만 아니라, 실행속도도 빠르다.
* `filter()`, `map()`을 이용해서도 만들 수 있지만, 속도가 빠르진 않다.
* Listcomp는 두 개 이상의 시퀀스를 만들 때도 사용할 수 있다, 이때는 개행으로 가독성을 확보해주자.
* Listcomp와 달리 genexp는 모든 항목이 들어있는 리스트를 생성하지 않고 한 번에 하나의 항목을 생성한다. 

# 2.3 튜플은 단순한 불변 리스트가 아니다

튜플은 불변 리스트로 사용할 수도 있지만, **필드명이 없는 레코드**로 사용할 수도 있다.

## 2.3.1 레코드로서의 튜플
* 튜플의 각 항목은 레코드의 필드 하나를 의미하며 항목의 위치가 곧 의미를 결정한다. 
* 튜플을 필드의 집합으로 사용하는 경우에는 항목 수가 고정되어 있고 항목의 순서가 중요. 

In [39]:
# 예제 2-7 레코드로 사용된 튜플
lax_coordinates = (33.9425, -118.408056)

# 도쿄에 대한 데이터 (지명, 년도, 백만 단위 인구수, 인구변화율, 제곱킬로미터 단위 면적)
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)

# (국가 코드, 여권 번호)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]

In [45]:
for passport in sorted(traveler_ids): # 리스트를 반복할 때 passport 변수가 각 튜플에 바인딩된다. 
    code, pass_num = passport
    print(f'{code}/{pass_num}')
print(type(passport))


BRA/CE342567
ESP/XDA205856
USA/31195855
<class 'tuple'>


레코드로서 사용하는 경우에 튜플을 정렬(sorted)해서 정보가 파괴되었다. 

In [46]:
for country, _ in traveler_ids: # for 루프트는 튜플의 각 항목을 어떻게 가져와야 하는지 알고 있다.
    print(country)              # (이 과정을 언패킹 이라고 함)

USA
BRA
ESP


## 2.3.2 튜플 언패킹
* 튜플 언패킹은 시퀀스 객체라면 모두 적용할 수 있다.
* 병렬 할당(parallel assignment)은 시퀀스 객체를 변수로 구성된 튜플에 할당하는 것을 말한다.

In [47]:
lax_coordinates = (33.9425, -118.408056)
latitude, longtitude = lax_coordinates # 튜플 언패킹

print('latitude   :', f'{latitude}')
print('longtitude :', f'{longtitude}')

latitude   : 33.9425
longtitude : -118.408056


튜플 언패킹을 이용하면 임시 변수를 사용하지 않고도 두 변수의 값을 서로 교환할 수 있다.

In [49]:
a = 2
b = 3

b, a = a, b

print('a :', f'{a}')
print('b :', f'{b}')

a : 3
b : 2


함수를 호출할 때 인수 앞에 *를 붙여 튜플을 언패킹할 수 있다.

In [50]:
divmod(20, 8)

(2, 4)

In [51]:
t = (20, 8)
divmod(*t)

(2, 4)

In [55]:
quotient, remainder = divmod(*t)
print(f'{quotient, remainder}')

(2, 4)


In [56]:
# 튜플 언패킹의 또다른 사용법
import os
_, filename = os.path.split('/home/luciano/.ssh/idrsa.pub')
filename

'idrsa.pub'

### 초과 항목을 잡기 위해 * 사용하기

In [57]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [58]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

## 2.3.4 명명된 튜플
`collections.namedtuple()` 함수는 필드명과 클래스명을 추가한 튜플의 서브클래스를 생성하는 팩토리 함수. 디버깅할 때 듀용하다.  
필드명이 클래스에 저장되므로 `namedtuple()`로 생성한 객체는 튜플과 동일한 크기의 메모리만 사용한다. 속성을 객체마다 존재하는 `__dict__` 에 저장하지 않으므로 일반적인 객체보다 메모리를 적게 사용한다. 

In [59]:
# 예제 2-9 namedtuple 정의하고 사용하기
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates') # 클래스명, 필드명의 리스트를 인자로 받음.
tokyo = City('Tokyo','JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [60]:
tokyo.population

36.933

In [61]:
tokyo.coordinates

(35.689722, 139.691667)

In [62]:
tokyo[1]

'JP'

namedtuple은 튜플에서 상속받은 속성 외에 `_files`, `_make`(iterable) 클래스 메서드, `_asdict()` 객체 메서드를 사용할 수 있다.

In [63]:
# 예제 2-10 named tuple의 속성과 메서드
City._fields # 클래스의 필드명을 담고 있는 튜플

('name', 'country', 'population', 'coordinates')

In [67]:
LatLong = namedtuple('Latlong', 'lat long')

delhi_data = ('Delhi NCR', 'IN', 21.935, LatLong(28.613889, 77.208889))

delhi = City._make(delhi_data) 
# ._make()는 반복형 객체로부터 naemdtuple을 만든다. Cit(*delhi_data)를 호출하는 코드와 동일한 역할을 한다.

delhi._asdict() # _asdict()는 namedtuple 객체에서 만들어진 collections.OrderedDict 객체를 반환한다.

OrderedDict([('name', 'Delhi NCR'),
             ('country', 'IN'),
             ('population', 21.935),
             ('coordinates', Latlong(lat=28.613889, long=77.208889))])

In [68]:
for key, value in delhi._asdict().items():
    print(key + ':', value)

name: Delhi NCR
country: IN
population: 21.935
coordinates: Latlong(lat=28.613889, long=77.208889)


## 2.3.5 불변 리시트로서의 튜플
튜플을 불변 리스트로 사용할 때, 튜플과 리스트가 얼마나 비슷한지 알고 있으면 도움이 된다. 
![표 2-1](./ch2_2.jpg)

## 2.4 슬라이싱

## 2.4.1 슬라이스와 범위 지정시에 마지막 항목이 포함되지 않는 이유

슬라이스와 범위 지정시에 마지막 항목을 포함하지 않는 관례 덕분에 아래와 같은 장점이 있다.
* 세 개의 항목을 생성하는 `range(3)`나 `my_list[:3]`처럼 중단점만 이용해서 슬라이스나 범위를 지정할 때 **길이를 계산하기 쉽다.**
* 시작점과 중단점을 모두 지정할 때도 길이를 계산하기 쉽다. **단지 중단점에서 시작점을 뺴면 된다.**
* 다음 예제에서 보는 것처럼, x 인덱스를 기준으로 **겹침 없이 시퀀스를 분할하기 쉽다.** 단지 my_list[:x]와 my_list[x:]로 지정하면 된다.

In [69]:
l = [10, 20, 30, 40, 50, 60]
l[:2] # 2번 인덱스에서 분할

[10, 20]

In [70]:
l[2:]

[30, 40, 50, 60]

## 2.4.2 슬라이스 객체
`s[a:b:c]`는 c 보폭(stride)만큼씩 항목을 건너뛰게 만든다. 음수이면 거꾸로 거슬러 올라간다.

In [71]:
s = 'bicycle'
s[::3]

'bye'

In [73]:
s[::-2]

'eccb'

* a:b:c 표기법은 인덱스 연산을 수행하는 []안에서만 사용할 수 있다.
* `slice(a, b, c)` 객체를 생성한다.
* `seq[start:stop:step]` 표현식을 평가하기 위해 `seq.__getitem__(slice(start, stop, step))`을 호출한다.

아래처럼 이름붙인 슬라이스 객체를 생성해서 가독성을 확보할 수 있다!

In [89]:
# 예제 2-11 단순 텍스트 파일 청구서의 행 항목들
invoice = """
0.....6...............................40........52...55........
1909  Pimoromi PiBrella                   $17.50    3    $52.50
1489  6mm Tactile Switch x20               $4.95    2     $9.90
1510  Panavise Jr. -Pv-201                $28.00    1    $28.00
1601  PiTFT Mini Kit 320x240              $34.95    1    $34.95
"""


SKU = slice(0,6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)
line_items = invoice.split('\n')[2:]

line_items

['1909  Pimoromi PiBrella                   $17.50    3    $52.50',
 '1489  6mm Tactile Switch x20               $4.95    2     $9.90',
 '1510  Panavise Jr. -Pv-201                $28.00    1    $28.00',
 '1601  PiTFT Mini Kit 320x240              $34.95    1    $34.95',
 '']

In [90]:
for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

  $17.50     Pimoromi PiBrella                 
   $4.95     6mm Tactile Switch x20            
  $28.00     Panavise Jr. -Pv-201              
  $34.95     PiTFT Mini Kit 320x240            
 


In [91]:
for item in line_items:
    print(item[ITEM_TOTAL])

  $52.50
   $9.90
  $28.00
  $34.95



## 2.4.3 다차원 슬라이싱과 생략 기호
* [] 연사자를 처리하는 `__getitem__()`과 `__setitem__` 특수 메서드는 `a[i, j]`에 들어 있는 인덱스들을 튜플로 받는다. (`a.__getitem__((i, j))`를 호출)
* 파이썬에 내장된 시퀀스형은 1차원이므로 단 하나의 인덱스나 슬라이스만 지원하고 튜플은 지원하지 않는다. 
* 세 개의 마침표(...)로 표현된 생략 기호는 Python parser에 의해 하나의 토큰으로 인식된다. 
* `f(a, ..., z)`처럼 함수의 인수나, `a[i:...]`처럼 슬라이스의 한 부분으로 전달할 수 있다.

## 2.4.4 슬라이스에 할당하기
* 할당문 왼쪽에 슬라이스 표기법을 사용하거나, `del` 문의 대상 객체로 지정함으로써
* 가변 시퀀스를 연결, 잘라 내기, 값 변경을 할 수 있다.

In [100]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [101]:
l[2:5] = [20, 30]
l

[0, 1, 20, 30, 5, 6, 7, 8, 9]

In [102]:
del l[5:7]
l

[0, 1, 20, 30, 5, 8, 9]

In [103]:
l[3::2] = [11, 22]

In [104]:
l

[0, 1, 20, 11, 5, 22, 9]

In [105]:
l[2:5] = 100
# 할당문의 대상이 슬라이스인 경우, 항목 하나만 할당하는 경우에도 할당문 오른쪽에는 iterable한 객체가 와야한다.

TypeError: can only assign an iterable

In [106]:
l[2:5] = [100]
l

[0, 1, 100, 22, 9]

In [107]:
l[3::2] = [11, 22]
# l[a:b:c]를 사용하는 경우, 오른쪽 할당 반복 객체에 갯수를 맞춰줘야한다!

ValueError: attempt to assign sequence of size 2 to extended slice of size 1

# 2.5 시퀀스에 덧셈과 곱셈 연산자 사용하기

a가 가변 항목을 담고 있을 때 `a * n`과 같은 표현식을 사용하려면 주의해야 한다.  
예를 들어, 리스트의 리스트를 초기화할 때 my_list = `[[]] * 3`으로 초기화하면 **동일한 내부 리스트에 대한 참조 세 개를 가진 리스트가 만들어지므로,** 원치 않는 결과가 나올 수 있다.

## 2.5.1 리스트의 리스트 만들기
내포된 리스트를 가진 리스트를 초기화해야 하는 경우, listcomp를 사용하는 것이 가장 좋다.

In [109]:
# 예제 2-12 길이가 3인 리스트 3개로 표현한 틱택토 보드
board = [['_'] * 3 for i in range(3)]
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [110]:
board[1][2] = 'X'
board

[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]

In [111]:
# 예제 2-13 동일한 리스트에 대한 세 개의 참조를 가진 리스트는 쓸모없다.
weird_board = [['_'] * 3] * 3 # 최상위 리스트가 동일한 내부 리스트에 대한 참조 세 개를 가진다.
weird_board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [112]:
# 동일한 참조를 가지기 때문에 이런 일이 발생한다..
weird_board[1][2] = 'O'
weird_board

[['_', '_', 'O'], ['_', '_', 'O'], ['_', '_', 'O']]

In [114]:
# 예제 2-13의 코드 작동 방식
row = ['_'] * 3
board = []
for i in range(3):
    board.append(row) # 동일한 행이 board 리스트에 세 번 추가된다.

In [115]:
# 예제 2-12의 listcomp의 코드 작동 방식
board = []
for i in range(3):
    row = ['_'] * 3
    board.append(row)

# 2.6 시퀀스의 복합 할당

* `+=`과 `*=`등의 복합 할당 연산자는 첫 번째 피연산자에 따라 상당히 다르게 작동한다. 
* `+=` 연산자가 작동하도록 만드는 특수 메서드는 `__iadd__()`다. 
* `__iadd__()` 메서드가 구현되어 있지 않으면, 파이썬은 `__add__()`를 대신 호출한다.

`a += b`  
* a가 `__iadd__()` 메서드를 구현하면 구현된 메서드가 호출된다.
* a가 list, bytearray, array.array 등 가변 시퀀스인 경우 a의 값이 변경된다.
* 그런데 a가 `__iadd__()` 메서드를 구현하지 않는 경우 `a += b` 표현식은 `a = a + b`가 되어 `a + b`를 먼저 평가하고 , **객체를 새로 생성한 후 a에 할당된다.**
* 즉, `__iadd__()` 메서드 구현 여부에 따라 a 변수가 가리키는 객체의 정체성이 바뀔 수도 있고 바뀌지 않을 수도 있다.
* 일반적으로 가변 시퀀스에 대해서는 `__iadd__()` 메서드를 구현해서 `+=` 연산자가 기존 객체의 내용을 변경하게 만드는 것이 좋다. 
* 불변 시퀀스의 경우에는 이 연산을 수행할 수 없다.
* `*=` 연산자의 경우 `__imul__()` 메서드를 통해 구현된다.

In [116]:
l = [1, 2, 3]
id(l) # 초기 리스트의 ID

1947438105224

In [117]:
l *= 2
l

[1, 2, 3, 1, 2, 3]

In [118]:
id(l) # 곱셈 연산을 수행한 후 새로운 항목이 추가된 리스트 객체는 기존 객체와 같다. 

1947438105224

In [119]:
t = (1, 2, 3)
id(t) # 초기 튜플의 ID

1947437251864

In [120]:
t *= 2
id(t) # 곱셈 연산을 수행한 후 새로운 튜플 객체가 만들어졌다.

1947449625000

예제에서 확인한 것처럼 **불변 시퀀스에 반복적으로 연결 연산을 수행하는 것은 비효율적**이다.   

하지만, str 객체의 작동 방식은 위와 다르며, str 객체는 메모리 안에 여분의 공간을 갖고 할당되므로,  
str 객체를 연결할 때 매번 전체 문자열을 다시 생성하지 않는다.

## 2.6.1 += 복합 할당 퀴즈

In [121]:
# 예제 2-14 퀴즈
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

In [122]:
t

(1, 2, [30, 40, 50, 60])

**이상하다;;**

![ch2_3.jpg](./ch2_3.jpg)

![ch2_4.jpg](./ch2_4.jpg)

www.pythontutor.com

**파이썬이 생성한 바이트코드를 살펴보자.**

In [123]:
import dis
dis.dis('s[a] += b')

  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


* `s[a]` 값을 스택의 꼭대기(TOS; Top Of Stack)에 놓는다.
* `TOS += b` 연산을 수행한다. `TOS`가 가변 객체([예제 2-15]에서의 리스트)를 가리키면 이 연산은 성공한다.
* `TOS`를 `s[a]`에 할당한다. `s`가 불변 객체([예제 2-15]에서의 t 튜플)면 이 연산은 실채한다.

### 교훈
* 가변 항목을 튜플에 넣는 것은 좋은 생각이 아니다.
* 복합 할당은 원자적인 연산이 아니다(앞의 예제에서 일부 연산이 수행된 후 예외가 발생했다.)
* 파이썬 바이트코드를 살펴보는 것은 그리 어렵지 않으며, 내부에서 어떤 일이 발생하고 있는지 살펴보는 데 도움이 된다.

# 2.7 list.sort()와 sorted() 내장 함수

* `list.sort()` 메서드는 사본을 만들지 않고 리스트 내부를 변경해서 정렬한다.
* `sort()` 메서드는 타깃 객체를 변경하고 새로운 리스트를 생성하지 않았음을 알려주기 위해 None을 반환한다.
* **이건 파이썬 API의 중요한 관례다!**
* 이와 반대로 `sorted()` 내장 함수는 새로운 리스트를 생성해서 반환한다.
* `sorted()` 함수는 불변 시퀀스 및 제너레이터를 포함해서 **반복 가능한 모든 객체를 인수로 받을 수 있다.**

**None을 반환하는 관례**
* 객체를 직접 변경했다고 알려주기 위해 None을 반환하는 관례는 메서드를 연결해서 호출할 수 없다는 단점이 있다.  
* 이와 반대로 str 객체의 메서드들처럼 새로운 객체를 반환하는 메서드는 플루언트 인터페이스 스타일로 메서드를 연결할 수 있다.  
* 메서드 체이닝은 플루언트 인터페이스 형태를 구현하는 하나의 기법이다.
  
**Fluent Interface** : 소프트웨어 공학에서 fluent interface는 소스 코드의 가독성을 산문과 유사하게 만드는 것이 목적이다. 특히 인터페이스 안에 도메인 특화 언어를 작성한다.

In [124]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
sorted(fruits) # 알파벳순으로 정렬된 정렬된 문자열을 담은 새로운 리스트 생성

['apple', 'banana', 'grape', 'raspberry']

In [125]:
fruits # 바뀌지 않았다.

['grape', 'raspberry', 'apple', 'banana']

In [126]:
sorted(fruits, reverse=True) # 역순으로 정렬

['raspberry', 'grape', 'banana', 'apple']

In [127]:
sorted(fruits, key=len) # 문장 길이에 따라 정렬. 길이가 같은 것은 그대로 순서 유지. 

['grape', 'apple', 'banana', 'raspberry']

In [128]:
sorted(fruits, key=len, reverse=True)

['raspberry', 'banana', 'grape', 'apple']

In [129]:
fruits

['grape', 'raspberry', 'apple', 'banana']

In [130]:
fruits.sort() # 아무것도 반환하지 않는다.

In [131]:
fruits # 원래 리스트 정렬

['apple', 'banana', 'grape', 'raspberry']

# 2.8 정렬된 시퀀스를 bisect로 관리하기
* `bisect()` : 이진 검색 알고리즘을 이용해서 시퀀스를 검색.
* `insort()` : 정렬된 시퀀스 안에 항목을 삽입.

## 2.8.1 bisect()로 검색하기
* `bisect(haystack, needle)`은 정렬된 시퀀스인 `haystack` 안에서 오름차순 정렬 상태를 유지한 채로 `needle`을 추가할 수 있는 위치를 찾아낸다.
* 해당 위치 앞에는 `needle`보다 같거나 작은 항목이 온다. 
* `bisect(haystack, needle)`의 결과값을 인덱스(index)로 사용해서 `haystack.insert(index, needle)`을 호출할 수 있지만,
* `insort()` 함수는 이 두 과정을 **더 빨리 처리한다.** 

`bisect()`  
1) 선택 인수인 `lo`와 `hi`를 사용하면 삽입할 때 검색할 시퀀스영역을 좁힐 수 있다.
* `lo`의 기본값은 `0`, `hi`의 기본값은 시퀀스의 `len()`이다.  

2) `bisect`은 실제로 `bisect_right()`함수의 별명이며, `bisect_left()`도 있다. 
* 리스트 안의 항목과 `needle` 값이 같을 때만 차이가 난다.

In [132]:
# 예제 2-17 정렬된 시퀀스에서 항목을 추가할 위치를 찾아내는 bisect
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle) # bisect을 이용해서 삽입 위치를 찾음.
        offset = position * '  |' # offsect에 비례한 수직 막대 패턴을 만듦.
        print(ROW_FMT.format(needle, position, offset)) # needle과 삽입 위치를 보여주는 행 출력
        
if __name__ == '__main__':
    if sys.argv[-1] == 'left': # 마지막 명령행 인수에 따라 사용할 bisect 함수 선택
        bisect_fn = bisect.bisect_left
    else:
        bisect_fn = bisect.bisect
        
    print('DEMO:', bisect_fn.__name__)
    print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
    demo(bisect_fn)

DEMO: bisect_right
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |29
23 @ 11      |  |  |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  5      |  |  |  |  |8 
 5 @  3      |  |  |5 
 2 @  1      |2 
 1 @  1      |1 
 0 @  0    0 


In [133]:
DEMO: bisect_left  
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30  
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31  
30 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |30  
29 @ 12      |  |  |  |  |  |  |  |  |  |  |  |29  
23 @  9      |  |  |  |  |  |  |  |  |23  
22 @  9      |  |  |  |  |  |  |  |  |22  
10 @  5      |  |  |  |  |10  
 8 @  4      |  |  |  |8  
 5 @  2      |  |5  
 2 @  1      |2  
 1 @  0    1  
 0 @  0    0  

SyntaxError: invalid syntax (<ipython-input-133-7a10db59c27b>, line 2)

In [134]:
# 예제 2-18 시험 점수를 입력받아 등급 문자를 반환하는 grade() 함수
def grade(score, breakpoints=[60, 70, 80, 90], grades='FDCBA'):
    i = bisect.bisect(breakpoints, score)
    return grades[i]

[grade(score) for score in [33, 99, 77, 70, 89, 90, 100]]

['F', 'A', 'C', 'C', 'B', 'A', 'A']

## 2.8.2 bisect.insort()로 삽입하기
정렬은 값비싼 연산이므로 시퀀스를 일단 정렬한 후에는 정렬 상태를 유지하는 것이 좋다.  
`insort(seq, item)`은 `seq`를 오름차순으로 유지한 채로 `item`을 `seq`에 삽입한다.

In [137]:
# 예제 2-19 시퀀스를 항상 정렬된 상태로 유지하는 insort()
import bisect
import random

SIZE = 7

random.seed(1729)

my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE*2)
    bisect.insort(my_list, new_item)
    print(f'{new_item:2d} -> {my_list}')

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]


# 2.9 리스트가 답이 아닐 때
* 배열은 모든 기능을 갖춘 float 객체 대신 C 언어의 배열과 마찬가지로 기계가 사용하는 형태로 표현된 **바이트 값만을 저장한다.**
* 리스트의 양쪽 끝에 항목을 추가하거나 삭제하면서 FIFO, LIFO 데이터 구조를 구현할 때는 **덱(deque)** 이 더 빠르다.

## 2.9.1 배열
* 리스트 안에 숫자만 들어있다면 배열이 리스트보다 훨씬 더 효율적이다.  
* 배열에 저장되는 항목의 문자 **타입코드(typecode)를 지정하여 메모리가 절약된다.**
* 배열은 `pop()`, `insert()`, `extend()`등을 포함해서 가변 시퀀스가 제공하는 모든 연산 + `frombytes()`와 `tofile()`메서드도 추가로 제공한다.

In [138]:
# 예제 -20 커다란 실수 배열의 생성, 저장, 로딩
from array import array
from random import random

floats = array('d', (random() for i in range(10**7))) # 실수타입 'd' 배열 생성
floats[-1]

0.5963321947530882

In [139]:
fp = open('floats.bin', 'wb')
floats.tofile(fp) # 배열을 이진 파일에 저장
fp.close()

floats2 = array('d')

fp = open('floats.bin', 'rb')
floats2.fromfile(fp, 10**7) # 이진 파일에서 천만 개의 숫자를 읽어온다. 
fp.close()

floats2[-1]

0.5963321947530882

In [140]:
floats2 == floats

True

* `float()` 내장 함수를 이용해서 파싱하면서 텍스트 파일에서 숫자를 읽어오는 것보다 **거의 60배 빠르다.**
* `array.tofile()` 메서드로 저장하는 것은 각 행마다 실수 하나씩 텍스트 파일에 저장하는 것보다 **약 7배 빠르다.**
* 배밀도 실수 천만 개를 저장한 이진 파일의 크기는 동일한 데이터를 저장한 텍스트 파일의 크기의 절반보다 적다.

**Tip)** 객체를 직렬화하는 `pickle` 모듈의 `pickle.dump()` 메서드도 빠르게 저장할 뿐만 아니라 사용자 정의 객체 등 거의 모든 내장 자료형을 처리한다.

## 2.9.2 메모리 뷰
메모리 뷰(`memoryview`) 내장 클래스는 공유 메모리 시퀀스형으로서 bytes를 복사하지 않고 배열의 슬라이스를 다룰 수 있게 해준다.

**언제 메모리 뷰를 사용해야 하는가?**  
*메모리 뷰는 본질적으로 (math를 포함하지 않은) 파이썬 자체에 들어 있는 NumPy 배열 구조체를 일반화한 것이다. 메모리 뷰는 PIL 이미지, SQLlite 데이터베이스, NumPy 배열 등 데이터 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다. 데이터셋이 커지는 경우 이것은 아주 중요한 기법이다.*

* `memoryview.cast()` 메서드는 바이트를 이동시키지 않고 여러 바이트로 된 데이터를 읽거나 쓰는 방식을 바꿀 수 있게 해준다. 
* `memoryview.cast()`는 또 다른 `memoryview` 객체를 반환하며 언제나 동일한 메모리를 공유한다.

In [141]:
# 예제 2-21 배열 항목 값의 바이트 중 하나를 변경하기
import array

numbers = array.array('h', [-2, -1, 0, 1, 2]) # signed integer 
memv = memoryview(numbers)

len(memv)

5

In [143]:
memv[0] # 다섯개 항목을 동일하게 본다. 

-2

In [144]:
memv_oct = memv.cast('B') # memv요소를 unsigned char('B')로 형변환한 memv_oct 생성

In [147]:
memv_oct.tolist() # 값을 조사하기 위해 list로 만듦
# unsigned char 타입으로 변환하면, 숫자, 부호가 추가 되어서 리스트 공간이 두 배로 늘어남.
# (-)부호는 255, +부호는 0으로 표현.

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

In [148]:
memv_oct[5] = 4

In [150]:
memv_oct[5]

4

In [151]:
numbers # 2 바이트 unsigned int의 최상위 바이트에서 4는 1024에 해당한다.

array('h', [-2, -1, 1024, 1, 2])

## 2.9.3 NumPy와 SciPy

pass

## 2.9.4 덱 및 기타 큐

* `append()`와 `pop()` 메서드를 사용해서 리스트를 큐로 사용할 수 있지만,   
리스트 왼쪽에 삽입 또는 삭제할 경우 **전체 리스트를 이동시켜야 하므로 처리 부담이 크다.**   


* 덱(`collections.deque`) 클래스는 큐의 양쪽 어디에서든 빠르게 삽입 및 삭제가 가능한 thread-safe 양방향 큐다.

* 덱은 최대 길이를 설정해서 제한된 항목만 유지할 수도 있으므로, 꽉찬 경우 새로운 항목을 삽입하면 반대쪽 항목이 제거된다.

In [152]:
# 예제 2-23 덱 이용하기
from collections import deque

# 선택적 인수 maxlen은 덱 객체가 수용할 수 있는 최대 항목 수를 설정한다.
# 덱 객체를 생성할 때 읽기 전용 속성인 maxlen을 설정한다.
dq = deque(range(10), maxlen=10)

dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [153]:
# 양수 인수를 받으면 오른쪽 끝에 있는 항목을 지정한 개수만큼 왼쪽 끝으로 이동
dq.rotate(3)

In [154]:
dq

deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [155]:
dq.rotate(-4)

In [156]:
dq

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

In [157]:
# 가득찬 덱에 항목을 추가하면 반대쪽 항목을 삭제한다.
dq.appendleft(-1)

In [158]:
dq

deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [159]:
dq.extend([11, 22, 33])

In [160]:
dq

deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33])

In [161]:
# extendleft(iter)는 iter 인수에서 생성되는 항목을 덱의 왼쪽에 하나씩 차례대로 추가한다,
# 그래서 항목이 역순으로 추가된 것임.
dq.extendleft([10, 20, 30, 40])

In [162]:
dq

deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8])